In [ ]:
!pip install langchain

In [ ]:
!pip install openai

In [ ]:
!pip install wikipedia

In [ ]:
!pip install numexpr

In [ ]:
!pip install langchain_community

In [ ]:
!pip install -U langchain-openai

In [ ]:
!pip install langgraph

## sample01

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-hXedXDpBrlTcFzTzuwsuT3BlbkFJNgLSTtQgf19RNc0i7K5b" #openai 키 입력

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 
                 model_name='gpt-4o-mini',  # 모델명
                )

In [ ]:
from langchain_core.messages import HumanMessage

response = llm.invoke([HumanMessage(content="hi!")])
response.content

## sample02

In [ ]:
# https://bcho.tistory.com/1427
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-hXedXDpBrlTcFzTzuwsuT3BlbkFJNgLSTtQgf19RNc0i7K5b" #openai 키 입력

#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 
                 model_name='gpt-4o-mini',  # 모델명
                )

from langchain.agents import load_tools
#from langchain.agents import initialize_agent
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
#from langchain import PromptTemplate
from langchain_core.prompts import PromptTemplate

# 도구 로드
tools = load_tools(["wikipedia", "llm-math"], llm=llm) #llm-math의 경우 나이 계산을 위해 사용


# 에이전트 초기화 (initialize_agent 대신 create_react_agent 사용)
# agent = initialize_agent(tools, 
#                          llm, 
#                          agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
#                          description='계산이 필요할 때 사용',
#                          verbose=True) 


# 도구 이름 추출
tool_names = ", ".join([tool.name for tool in tools])
# 초기화된 빈 작업 기록 (Scratchpad)
agent_scratchpad = ""

# 프롬프트 템플릿 생성
#ChatGPT
#https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/generative-ai/question-answering.ipynb
prompt = PromptTemplate.from_template(
    "You are an AI assistant. Answer the question using the following format:\n"
    "Thought: [What you are thinking]\n"
    "Action: [The action you will take]\n"
    "Action Input: [The input required for the action]\n"
    "If you have the final answer, respond with:\n"
    "Final Answer: [Your answer]\n\n"
    "Question: {input}\n\n"
    "Tools: {tools}\n\n"
    "Tool Names: {tool_names}\n\n"
    "Agent Scratchpad: {agent_scratchpad}"
)

# How to migrate from legacy LangChain agents to LangGraph https://python.langchain.com/v0.2/docs/how_to/migrate_agent/
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt.partial(
        tools=tools,
        tool_names=tool_names,
        agent_scratchpad=agent_scratchpad
    )
)

# 에이전트 실행기 초기화
# Build an Agent with AgentExecutor (Legacy) https://python.langchain.com/v0.2/docs/how_to/agent_executor/
# https://python.langchain.com/v0.1/docs/modules/agents/how_to/handle_parsing_errors/
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

#에이전트 실행
#agent.run("에드 시런이 태어난 해는? 2024년도 현재 에드 시런은 몇 살?")
result = agent_executor.invoke({"input": "에드 시런이 태어난 해는? 2024년도 현재 에드 시런은 몇 살?"})
print(result["output"])

## sample03

In [ ]:
# https://bcho.tistory.com/1427
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-hXedXDpBrlTcFzTzuwsuT3BlbkFJNgLSTtQgf19RNc0i7K5b" #openai 키 입력

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 
             model_name='gpt-4o-mini',  # 모델명
            )

from langchain.agents import load_tools
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain_core.prompts import PromptTemplate

# 도구 로드
tools = load_tools(["wikipedia", "llm-math"], llm=llm) #llm-math의 경우 나이 계산을 위해 사용

# 도구 이름 추출
tool_names = ", ".join([tool.name for tool in tools])
# 초기화된 빈 작업 기록 (Scratchpad)
agent_scratchpad = ""

# 프롬프트 템플릿 생성
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)
search_agent = create_react_agent(llm,tools,prompt)

agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)
response = agent_executor.invoke({"input": "Where is the hometown of the 2007 US PGA championship winner and his score?"})
print(response)


## sample04

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/migrate_agent/#iterating-through-steps
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-hXedXDpBrlTcFzTzuwsuT3BlbkFJNgLSTtQgf19RNc0i7K5b" #openai 키 입력

from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = load_tools(["wikipedia", "llm-math"], llm=llm) #llm-math의 경우 나이 계산을 위해 사용

agent = create_tool_calling_agent(llm, tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "안중근 의사가 태어 난 해는 언제인가요? 2024년도 현재 살아 있다면 안중근 의사는 몇 살일까요?"
result = agent_executor.invoke({"input": query})
print(result["output"])

query = "아이즈원 김민주가 태어 난 해는 언제인가요? 2024년도 현재 몇 살일까요?"
for step in agent_executor.stream({"input": query}):
    print(step)